In [1]:
import pandas as pd

In [2]:
csv_path = r"C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data7.csv"
df = pd.read_csv(csv_path, index_col=0)

In [3]:
# df

In [4]:
y = df['change']
x=df.drop(['change'],axis=1)

In [5]:
# x

In [6]:
type(y)

pandas.core.series.Series

In [7]:
y.value_counts()

change
1    56
0    46
Name: count, dtype: int64

# Filter Method

In [8]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Mutual Information
selector = SelectKBest(mutual_info_classif, k=10)  # Select top 10 features
X_selected = selector.fit_transform(x,y)

In [9]:
selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = x.columns[selected_feature_indices].tolist()

In [10]:
selected_feature_names

['original_shape_MajorAxisLength',
 'original_shape_Maximum2DDiameterRow',
 'original_shape_Maximum3DDiameter',
 'original_shape_SurfaceArea',
 'original_shape_SurfaceVolumeRatio',
 'original_gldm_DependenceNonUniformity',
 'original_glrlm_GrayLevelNonUniformity',
 'original_glrlm_RunLengthNonUniformity',
 'original_glszm_GrayLevelNonUniformity',
 'original_ngtdm_Coarseness']

In [11]:
x_filter=x[selected_feature_names]

### only 4 features were common
[ 'original_shape_Maximum2DDiameterRow',
 'original_shape_Maximum3DDiameter',
 'original_shape_SurfaceArea',
 'original_ngtdm_Coarseness']

# Wrapper Method

In [12]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Recursive Feature Elimination
model = RandomForestClassifier()
rfe = RFE(estimator=model, n_features_to_select=10)
X_rfe = rfe.fit_transform(x, y)

In [13]:
selected_feature_indices_rfe = rfe.get_support(indices=True)
selected_feature_names_rfe = x.columns[selected_feature_indices_rfe].tolist()
selected_feature_names_rfe

['original_shape_MajorAxisLength',
 'original_shape_Maximum2DDiameterRow',
 'original_shape_SurfaceVolumeRatio',
 'original_glcm_Correlation',
 'original_glcm_MCC',
 'original_gldm_DependenceNonUniformity',
 'original_gldm_GrayLevelNonUniformity',
 'original_glszm_LargeAreaHighGrayLevelEmphasis',
 'original_ngtdm_Coarseness',
 'original_ngtdm_Strength']

#### 6 feactures matched with filter method
#### 5 feactures matched with Anirrudh's results

In [14]:
x_wrapper=x[selected_feature_names_rfe]

In [15]:
# x_wrapper

# Wrapper Training

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import numpy as np

In [17]:
X_train, X_test , y_train, y_test = train_test_split(x_wrapper, y, test_size=0.2, random_state=42)

In [18]:
y_train.value_counts()

change
1    42
0    39
Name: count, dtype: int64

In [19]:
len(X_train), len(X_test)

(81, 21)

In [23]:
# Random Forest

rf_classifier = RandomForestClassifier(n_estimators=300, random_state=1)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8095238095238095
              precision    recall  f1-score   support

           0       0.71      0.71      0.71         7
           1       0.86      0.86      0.86        14

    accuracy                           0.81        21
   macro avg       0.79      0.79      0.79        21
weighted avg       0.81      0.81      0.81        21



In [24]:
y_prob = rf_classifier.predict_proba(x_wrapper)
y_prob[:, 1]

array([0.99333333, 0.97      , 0.01      , 0.87666667, 0.23      ,
       0.31666667, 0.86      , 0.01666667, 0.02333333, 0.26666667,
       0.87333333, 0.24333333, 0.52      , 0.13333333, 0.03333333,
       0.83      , 0.91      , 0.8       , 0.02      , 0.12      ,
       0.99666667, 0.05333333, 0.07      , 0.02666667, 0.79333333,
       0.13666667, 0.98666667, 0.22333333, 0.17333333, 0.00666667,
       0.83      , 0.57      , 0.98333333, 0.56      , 0.96666667,
       0.00666667, 0.88      , 0.10666667, 1.        , 0.84666667,
       0.99      , 0.94      , 0.99333333, 0.21666667, 0.15      ,
       0.30333333, 0.00333333, 0.97      , 0.78666667, 1.        ,
       0.86333333, 0.97      , 0.97333333, 0.83      , 0.25666667,
       0.21666667, 0.05666667, 0.01      , 0.07666667, 0.05666667,
       0.26666667, 0.93666667, 0.51      , 0.78666667, 0.05333333,
       0.98      , 0.89666667, 0.98333333, 0.93333333, 0.70666667,
       1.        , 0.07      , 0.18333333, 0.97666667, 0.33   

In [118]:
import pandas as pd
import numpy as np

y = y.to_numpy() if isinstance(y, pd.Series) else y
y_prob = y_prob.to_numpy() if isinstance(y_prob, pd.DataFrame) else y_prob

y = y.ravel()
y_prob = y_prob[:, 1].ravel()  # Assuming y_prob is shape (N, 2)
predicted_labels = [1 if prob >= 0.5 else 0 for prob in y_prob]

df = pd.DataFrame({
    "Original": y,
    "Probability": y_prob,
    "Predicted": predicted_labels
})

file_path = "predictions.csv"
df.to_csv(file_path, index=False)

print(f"File saved as: {file_path}")

File saved as: predictions.csv


In [25]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.7142857142857143
              precision    recall  f1-score   support

           0       0.56      0.71      0.62         7
           1       0.83      0.71      0.77        14

    accuracy                           0.71        21
   macro avg       0.69      0.71      0.70        21
weighted avg       0.74      0.71      0.72        21



In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_wrapper, y, test_size=0.2, random_state=1)

sgd_classifier = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)

# Wrap training process in tqdm for progress bar
batch_size = 2  # Define the batch size
n_batches = len(X_train) // batch_size + (1 if len(X_train) % batch_size > 0 else 0)

for _ in tqdm(range(n_batches), desc="Training SVM Model in batches"):
    # Train in batches
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]
        sgd_classifier.partial_fit(X_batch, y_batch, classes=np.unique(y_train))

# Predict on the test set
y_pred = sgd_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Training SVM Model in batches: 100%|██████████| 41/41 [00:03<00:00, 10.28it/s]

Accuracy: 0.3333333333333333
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         7
           1       0.00      0.00      0.00        14

    accuracy                           0.33        21
   macro avg       0.17      0.50      0.25        21
weighted avg       0.11      0.33      0.17        21




c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
# XGBoost

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(x_wrapper, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss') #eval_metric added
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

XGBoost Accuracy: 0.7142857142857143
              precision    recall  f1-score   support

           0       0.57      0.57      0.57         7
           1       0.79      0.79      0.79        14

    accuracy                           0.71        21
   macro avg       0.68      0.68      0.68        21
weighted avg       0.71      0.71      0.71        21



c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\xgboost\training.py:183: UserWarning: [01:13:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [26]:
# !pip install xgboost

# Filter method training

In [28]:
X_train, X_test , y_train, y_test = train_test_split(x_filter, y, test_size=0.2, random_state=42)

In [31]:
# Random Forest

rf_classifier = RandomForestClassifier(n_estimators=300, random_state=1)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.7142857142857143
              precision    recall  f1-score   support

           0       0.56      0.71      0.62         7
           1       0.83      0.71      0.77        14

    accuracy                           0.71        21
   macro avg       0.69      0.71      0.70        21
weighted avg       0.74      0.71      0.72        21



In [32]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.7142857142857143
              precision    recall  f1-score   support

           0       0.56      0.71      0.62         7
           1       0.83      0.71      0.77        14

    accuracy                           0.71        21
   macro avg       0.69      0.71      0.70        21
weighted avg       0.74      0.71      0.72        21



In [33]:
sgd_classifier = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)

# Wrap training process in tqdm for progress bar
batch_size = 2  # Define the batch size
n_batches = len(X_train) // batch_size + (1 if len(X_train) % batch_size > 0 else 0)

for _ in tqdm(range(n_batches), desc="Training SVM Model in batches"):
    # Train in batches
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]
        sgd_classifier.partial_fit(X_batch, y_batch, classes=np.unique(y_train))

# Predict on the test set
y_pred = sgd_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Training SVM Model in batches:   0%|          | 0/41 [00:00<?, ?it/s]

Training SVM Model in batches: 100%|██████████| 41/41 [00:03<00:00, 10.62it/s]

Accuracy: 0.3333333333333333
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         7
           1       0.00      0.00      0.00        14

    accuracy                           0.33        21
   macro avg       0.17      0.50      0.25        21
weighted avg       0.11      0.33      0.17        21




c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Acer\anaconda3\envs\3dVS\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
